In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
import os
from sqlalchemy import create_engine

In [2]:
os.makedirs("/home/omar/spark-temp-batch", exist_ok=True)

# Crée une session Spark indépendante pour le batch
spark = SparkSession.builder \
    .appName("BatchApp") \
    .config("spark.local.dir", "/home/omar/spark-temp-batch") \
    .config("spark.ui.port", "4051") \
    .config("spark.jars","/home/omar/libs/postgresql-42.6.0.jar") \
    .getOrCreate()


25/06/10 21:27:51 WARN Utils: Your hostname, omar-VirtualBox resolves to a loopback address: 127.0.0.1; using 192.168.0.181 instead (on interface enp0s3)
25/06/10 21:27:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/06/10 21:27:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/10 21:27:52 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [3]:


# Remplace les valeurs ci-dessous par les tiennes
df = spark.read.format("jdbc").option("url", "jdbc:postgresql://192.168.0.135:5432/telecom_db") \
    .option("dbtable", "customers") \
    .option("user", "postgres") \
    .option("password", "123") \
    .option("driver", "org.postgresql.Driver") \
    .load()



In [5]:
df.show()

+--------------------+------------+-----------------+------------+---------------+----------+-------------+----------+-------------+
|         customer_id|      msisdn|    customer_name|rate_plan_id|activation_date|    status|       region|is_student|loyalty_start|
+--------------------+------------+-----------------+------------+---------------+----------+-------------+----------+-------------+
|567ce95c-6719-4f6...|212616417735|     Omar Belkadi|           9|     2022-04-27| suspended|   Errachidia|     false|   2022-08-11|
|8d66974a-6e29-419...|212618907384|     Imane Jabari|           3|     2021-01-09|terminated|  Beni Mellal|     false|         NULL|
|1dd6eb30-b835-46d...|212675899785|     Ahmed Lahcen|           4|     2024-08-02|terminated|rural_central|     false|         NULL|
|2a5cf491-d076-453...|212675676874|    Youssef Dahbi|           1|     2021-12-07|    active|international|     false|         NULL|
|50dc9474-710a-46d...|212699322434|      Imane Dahbi|           2|   

In [6]:

db_url = "jdbc:postgresql://192.168.0.135:5432/telecom_db"
tables = {
    "normalized_events": "clean_stream",
    "customers": "customers",
    "product_catalog": "product_catalog",
    "product_rates": "product_rates",
    "rate_plans": "rate_plans",
    "billing_cycles": "billing_cycles",
    "promotions": "promotions",
    "tax_rules": "tax_rules"
}
properties = {
    "user": "postgres",
    "password": "123",
    "driver": "org.postgresql.Driver"
}
dfs = {}
for name, table in tables.items():
    dfs[name] = spark.read.jdbc(
        url=db_url,
        table=table,
        properties=properties
    ).cache()

In [9]:
dfs["normalized_events"].show()

+-----------+-------------------+------------+------------+---------+-----------+-------+------------+--------------------+--------------+----------+----------+--------------------+--------------------+-------------------+------------+-----------------+----------+
|record_type|          timestamp|   caller_id|   callee_id|sender_id|receiver_id|user_id|duration_sec|session_duration_sec|data_volume_mb|   cell_id|technology|normalized_caller_id|normalized_callee_id|normalized_duration|      msisdn|normalized_volume|error_flag|
+-----------+-------------------+------------+------------+---------+-----------+-------+------------+--------------------+--------------+----------+----------+--------------------+--------------------+-------------------+------------+-----------------+----------+
|      voice|2025-05-01 00:59:59|212685152914|212748091911|     NULL|       NULL|   NULL|         228|                NULL|          NULL|Al Hoceïma|        5G|        212685152914|        212748091911|   

In [11]:
# Filtrer uniquement les clients actifs et postpaid
active_customers = dfs["customers"].filter(
    (col("status") == "active") 
)

# Jointure entre événements et clients
events_customers = dfs["normalized_events"].join(
    active_customers,
    "msisdn",
    "inner"
)

# Ajouter le type de service basé sur record_type
events_customers = events_customers.withColumn(
    "service_type",
    when(col("record_type") == "voice", "voice")
    .when(col("record_type") == "sms", "sms")
    .when(col("record_type") == "data", "data")
)

In [17]:


# Étape 1 : jointure avec product_catalog
events_with_products = events_customers.join(
    dfs["product_catalog"].alias("catalog"),
    "service_type",
    "left"
)

# Étape 2 : jointure avec product_rates
rated_events = events_with_products.alias("events").join(
    dfs["product_rates"].alias("rates"),
    (col("events.rate_plan_id") == col("rates.rate_plan_id")) &
    (col("events.product_code") == col("rates.product_code")),
    "left"
)



In [19]:
rated_events = rated_events.withColumn(
    "base_cost",
    when(col("service_type") == "voice", col("duration_sec") * col("unit_price"))
    .when(col("service_type") == "sms", lit(1) * col("unit_price"))  # 1 SMS = 1 événement
    .when(col("service_type") == "data", col("data_volume_mb") * col("unit_price"))
)

# Application des modificateurs temporels (exemple simplifié)
rated_events = rated_events.withColumn(
    "time_multiplier",
    when(hour(col("timestamp")).between(9, 17), 1.2).otherwise(1.0)  # Peak/Off-peak
).withColumn(
    "adjusted_cost", 
    col("base_cost") * col("time_multiplier")
)

In [23]:
rated_events.printSchema()

root
 |-- service_type: string (nullable = true)
 |-- msisdn: string (nullable = true)
 |-- record_type: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- caller_id: string (nullable = true)
 |-- callee_id: string (nullable = true)
 |-- sender_id: string (nullable = true)
 |-- receiver_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- duration_sec: integer (nullable = true)
 |-- session_duration_sec: double (nullable = true)
 |-- data_volume_mb: double (nullable = true)
 |-- cell_id: string (nullable = true)
 |-- technology: string (nullable = true)
 |-- normalized_caller_id: string (nullable = true)
 |-- normalized_callee_id: string (nullable = true)
 |-- normalized_duration: double (nullable = true)
 |-- normalized_volume: double (nullable = true)
 |-- error_flag: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- rate_plan_id: integer (nullable = true)
 |-- activation_da

In [25]:
from pyspark.sql.types import StructType

# Récupérer le schéma du DataFrame existant
schema = rated_events.schema

# Créer un DataFrame vide avec le même schéma
df_vide = spark.createDataFrame([], schema)

In [26]:
colonnes = ["adjusted_cost", "base_cost", "customer_name","customer_id"]
df_new_rated = rated_events.select(colonnes)

In [ ]:
df_new_rated.show(100)

In [31]:
df_new_rated.groupBy("customer_id", "customer_name").agg(
    sum("base_cost").alias("total_base_cost")
).show()

+--------------------+-----------------+------------------+
|         customer_id|    customer_name|   total_base_cost|
+--------------------+-----------------+------------------+
|e774b96d-9d1a-446...|   Fatima Idrissi| 2835.056100000001|
|c84f754a-e615-4ef...|      Nora Lahcen| 2407.335771999998|
|f20443c6-bcc7-408...|  Youssef Idrissi|1322.6277540000008|
|54f1469d-e0da-4be...|      Nora Gharbi|2884.7758840000015|
|58060ca1-6e54-486...|   Fatima Belkadi|2498.4831000000013|
|5fdcef74-3141-43b...|   Youssef Gharbi|2035.4662440000004|
|399e817e-6ff4-438...|    Salma Idrissi|1822.9705999999994|
|d48c8867-9e02-4e4...|Khadija El Amrani|2666.7378000000017|
|0dd992d5-7c3a-4ae...|     Fatima Fassi|1982.1301279999998|
|fe9df7dd-8c28-459...|    Imane Choukri| 4457.073206000002|
|9aa188f7-58d3-482...|    Hassan Gharbi|2797.3998759999977|
|ff86d842-0d96-43b...|      Imane Haqiq|         3191.7159|
|8f388b1e-b65b-49b...|       Nora Dahbi|3140.8918990000006|
|0fc63ea4-209e-4be...|   Abdel Mansouri|

# Étape 6: Gestion des erreurs et statuts

In [34]:
rated_events = rated_events.withColumn(
    "rating_status",
    when(col("events.product_code").isNull(), "unmatched")
    .when(col("events.duration_sec").isNull() & (col("events.service_type") == "voice"), "error")
    .when(col("events.data_volume_mb").isNull() & (col("events.service_type") == "data"), "error")
    .otherwise("rated")
)

# Filtrage
error_events = rated_events.filter(col("rating_status") != "rated")
valid_events = rated_events.filter(col("rating_status") == "rated")

# Étape 7: Application des promotions

In [35]:
valid_events = valid_events.withColumn(
    "discount_amount",
    when(col("is_student"), col("adjusted_cost") * 0.1).otherwise(0)
).withColumn(
    "final_cost",
    col("adjusted_cost") - col("discount_amount")
)

# Appliquer les promotions de fidélité (>12 mois)
valid_events = valid_events.withColumn(
    "loyalty_discount",
    when(months_between(current_date(), col("loyalty_start")) > 12, 
         col("final_cost") * 0.05).otherwise(0)
).withColumn(
    "final_cost",
    col("final_cost") - col("loyalty_discount")
)

In [37]:
valid_events.printSchema()

root
 |-- service_type: string (nullable = true)
 |-- msisdn: string (nullable = true)
 |-- record_type: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- caller_id: string (nullable = true)
 |-- callee_id: string (nullable = true)
 |-- sender_id: string (nullable = true)
 |-- receiver_id: string (nullable = true)
 |-- user_id: string (nullable = true)
 |-- duration_sec: integer (nullable = true)
 |-- session_duration_sec: double (nullable = true)
 |-- data_volume_mb: double (nullable = true)
 |-- cell_id: string (nullable = true)
 |-- technology: string (nullable = true)
 |-- normalized_caller_id: string (nullable = true)
 |-- normalized_callee_id: string (nullable = true)
 |-- normalized_duration: double (nullable = true)
 |-- normalized_volume: double (nullable = true)
 |-- error_flag: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- rate_plan_id: integer (nullable = true)
 |-- activation_da

# Étape 8: Enregistrement des résultats

In [54]:
# Sélection des colonnes finales
final_rated_events = valid_events.select(
    "msisdn",
    "customer_id",
    "customer_name",
    "record_type",
    "timestamp",
    "service_type",
    "base_cost",
    "adjusted_cost",
    "discount_amount",
    "final_cost",
    "rating_status",
    "`events`.`rate_plan_id`",
    "`events`.`product_code`"
)


In [55]:
final_rated_events.printSchema()

root
 |-- msisdn: string (nullable = true)
 |-- customer_id: string (nullable = true)
 |-- customer_name: string (nullable = true)
 |-- record_type: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- service_type: string (nullable = true)
 |-- base_cost: double (nullable = true)
 |-- adjusted_cost: double (nullable = true)
 |-- discount_amount: double (nullable = true)
 |-- final_cost: double (nullable = true)
 |-- rating_status: string (nullable = false)
 |-- rate_plan_id: integer (nullable = true)
 |-- product_code: string (nullable = true)



In [69]:
df_with_month = final_rated_events.withColumn("month", month(col("timestamp")))

# Puis agrégation en groupant aussi par "month" si tu souhaites regrouper par mois
billing_df = df_with_month.groupBy("customer_id", "customer_name", "month").agg(
    sum("final_cost").alias("total_final_cost"),
    sum("base_cost").alias("before_promo")
)

In [73]:
df_with_month.show(100)


+------------+--------------------+--------------+-----------+-------------------+------------+---------+-------------+------------------+------------------+-------------+------------+----------------+-----+
|      msisdn|         customer_id| customer_name|record_type|          timestamp|service_type|base_cost|adjusted_cost|   discount_amount|        final_cost|rating_status|rate_plan_id|    product_code|month|
+------------+--------------------+--------------+-----------+-------------------+------------+---------+-------------+------------------+------------------+-------------+------------+----------------+-----+
|212685152914|f6a883bb-1afb-47a...|Ahmed Mansouri|      voice|2025-05-01 00:59:59|       voice|     5.13|         5.13|               0.0|              5.13|        rated|           2|     V_CORP_TIME|    5|
|212685152914|f6a883bb-1afb-47a...|Ahmed Mansouri|      voice|2025-05-01 00:59:59|       voice|   3.0552|       3.0552|               0.0|            3.0552|        rat